# **Problem 1: WhiteBox Attacks Using FoolBox**

Install Foolbox

In [ ]:
!pip3 install foolbox

## Getting Model

In [ ]:
import torch as t
import torchvision as tv
import foolbox as fb

model = tv.models.resnet18(pretrained = True)
prep = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
bounds = (0,1)
fmodel = fb.PyTorchModel(model,bounds=bounds,preprocessing=prep)

/usr/local/lib/python3.7/dist-packages/foolbox/models/pytorch.py:37: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  "The PyTorch model is in training mode and therefore might"


Data Set

In [ ]:
images,labels = fb.utils.samples(fmodel,dataset='imagenet',batchsize=16)

Tools that will be used for Data Gathering

In [ ]:
#stores the 4 data fields into lists
attackType = []
pnormdist = []
succrate = []
timecost = []

#function for appending all items at once
def appenditems(at,pd,sr,tc):
  attackType.append(at)
  pnormdist.append(pd)
  succrate.append(sr)
  timecost.append(tc)
  return

#function to convert seconds to minutes
import time
def convert(seconds): 
    min, sec = divmod(seconds, 60)
    return "%02d:%02d" % (min, sec)

## Attack Model

In [ ]:
#accuracy
fb.utils.accuracy(fmodel, images, labels)

0.875

In [ ]:
#attack function
import numpy as np
epsilons = np.linspace(0.0,0.005, num=5)

def flatlist(l):
    flatlist = []
    for e in l:
      flatlist.extend(e)
    return flatlist

def attk(at,pd,attack):
  ti = time.time()
  raw, clipped, is_adv = attack(fmodel,images,labels,epsilons=epsilons)
  tc = convert(time.time()-ti)
  fl = flatlist(is_adv.tolist())
  sr = fl.count(True)/len(fl)
  appenditems(at,pd,sr,tc)
  return

In [ ]:
#Deep Fool Attacks
attk('Deep Fool','2',fb.attacks.L2DeepFoolAttack())
attk('Deep Fool','Inf',fb.attacks.LinfDeepFoolAttack())

In [ ]:
#Basic Iterative Attacks
attk('Basic Iterative','1',fb.attacks.L1BasicIterativeAttack())
attk('Basic Iterative','2',fb.attacks.L2BasicIterativeAttack())
attk('Basic Iterative','Inf',fb.attacks.LinfBasicIterativeAttack())

In [ ]:
#Fast Gradiant Attacks
attk('Fast Gradiant','1',fb.attacks.L1FastGradientAttack())
attk('Fast Gradiant','2',fb.attacks.L2FastGradientAttack())
attk('Fast Gradiant','Inf',fb.attacks.LinfFastGradientAttack())

In [ ]:
#Projected Griadiant Descent Attacks
attk('Projected Gradiant Descent','1',fb.attacks.L1ProjectedGradientDescentAttack())
attk('Projected Gradiant Descent','2',fb.attacks.L2ProjectedGradientDescentAttack())
attk('Projected Gradiant Descent','Inf',fb.attacks.LinfProjectedGradientDescentAttack())

## Finalize Table

In [ ]:
import pandas as pd
df = pd.DataFrame({'Attack Type' : attackType,
                   'P-norm Distance' : pnormdist,
                   'Success Rate' : succrate,
                   'Time Cost (minutes)' : timecost
})
df

,Attack Type,P-norm Distance,Success Rate,Time Cost (minutes)
0,Deep Fool,2,0.1250,33:01
1,Deep Fool,Inf,0.7125,20:49
2,Basic Iterative,1,0.1250,03:53
3,Basic Iterative,2,0.1250,03:48
4,Basic Iterative,Inf,0.6375,03:48
5,Fast Gradiant,1,0.1250,00:29
6,Fast Gradiant,2,0.1250,00:29
7,Fast Gradiant,Inf,0.5000,00:28
8,Projected Gradiant Descent,1,0.1250,18:33
9,Projected Gradiant Descent,2,0.1250,18:24
